In [1]:
# ! conda install -c conda-forge imbalanced-learn
# ! pip install nose
# ! pip install imbalanced-ensemble           

In [11]:
import re

def read_dot_dat_file(path):
    datContent = [i.strip().split() for i in open(path).readlines()]
    r = re.compile("@inputs.*")
    _at_data = datContent.index(['@data'])
    assert datContent[0][0] == '@relation'
    assert datContent[_at_data-1][0] == '@outputs'
    assert datContent[_at_data-2][0] == '@inputs'
    print(datContent[_at_data-3][2:])
    assert len(datContent[_at_data-3][2:]) == 2   # Two Class

    col_names = datContent[_at_data-2][1:]
    col_names.append(datContent[_at_data-1][1])
    
    df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=r', ', engine='python')
    # df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=", ", engine='python')

    class1 = datContent[_at_data-3][2:][0].replace("{","").replace(",","")
    class2 = datContent[_at_data-3][2:][1].replace("}","").replace(",","")

    df['Class'] = df['Class'].replace({class1: 1, class2: -1})
    
    return df



# temp

In [92]:
import pandas as pd 
import numpy as np
import os

base_path = 'Datasets/'
# need_to_convert = ['wisconsin-5-fold','new-thyroid2-5-fold','new-thyroid1-5-fold']
# need_to_convert = ['yeast-1_vs_7-5-fold', 'led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold', 'ecoli-0-3-4-6_vs_5-5-fold', 'abalone19-5-', 'abalone19-5-fold']
need_to_convert = ['ecoli-0-1_vs_5-5-fold']
for needed in need_to_convert:
    for datFile in glob.glob(base_path+needed+'/*.dat'):
        print(datFile)
        df = read_dot_dat_file(datFile)
        df.to_excel(base_path+needed+'/'+datFile.split('/')[-1].split('.')[0]+'.xlsx')


Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-4tst.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-3tra.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-5tra.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-4tra.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-3tst.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-5tst.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-1tra.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-1tst.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-2tra.dat
['{1,', '-1}']
Datasets/ecoli-0-1_vs_5-5-fold/ecoli-0-1_vs_5-5-2tst.dat
['{1,', '-1}']


In [12]:
import time
import os
import glob
import numpy as np
import random  
import pandas as pd 
import random

from sklearn.tree import DecisionTreeClassifier 

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc,f1_score,matthews_corrcoef
from imblearn.metrics import geometric_mean_score


from imblearn.ensemble import RUSBoostClassifier
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier
from maatpy.classifiers import AdaCost


In [87]:
base_path = 'Datasets/'
datasets  = os.listdir(base_path) 
classifiers = {"RUS": RUSBoostClassifier(random_state=0, algorithm='SAMME', estimator=DecisionTreeClassifier(max_depth=10)),
              "SMOTE": SMOTEBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10), n_estimators = 100, algorithm='SAMME', random_state=0),
              "Ada1": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adac1',random_state=0),
              "AdaCost": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adacost',random_state=0),}


def clasify(dataset, classifier):
    mcc = []
    f1 = []
    auc_a = []
    gmean = []
    times = []
    y_preds = []
    traFiles = sorted(glob.glob(base_path+dataset+'/*tra.xlsx'))
    tstFiles = sorted(glob.glob(base_path+dataset+'/*tst.xlsx'))
    for traPath, tstPath in zip(traFiles, tstFiles):
        print(traPath)
        
        df_train = pd.read_excel(traPath)
        df_test = pd.read_excel(tstPath)

        x_train= df_train.iloc[:, 0:-1]
        y_train = df_train.iloc[:, -1]
        x_test= df_test.iloc[:, 0:-1]
        y_test = df_test.iloc[:, -1]
        
        #####
        # for some dataset get error Unknown label type: 'unknown'
        y_train = y_train.astype('int')
        y_test = y_test.astype('int')
        
        st = time.time()
        clf = classifiers[classifier]

        clf.fit(x_train, y_train)

        y_pred = clf.predict(x_test) 
        et = time.time()
        y_preds.append(y_pred)
        # compute error
        mcc.append(matthews_corrcoef(y_test, y_pred))
        #--------------------------------
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auc_a.append(auc(fpr, tpr))
        #--------------------------------
        f1.append(f1_score(y_test, y_pred))
        #--------------------------------
        gmean.append(geometric_mean_score(y_test, y_pred, labels=[1, -1]))
        
        #time of train and test
        times.append(et - st)
        
    return {"mcc": mcc, "auc": auc_a, "f1": f1, "gmean": gmean, "exe_time": times, "y_pred": y_pred}

aaaaaa =  clasify(datasets[0], 'Ada1')
aaaaaa

Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-1tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-2tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-3tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-4tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-5tra.xlsx


{'mcc': [0.0, 0.0, 0.0, 0.0, 0.0],
 'auc': [0.5, 0.5, 0.5, 0.5, 0.5],
 'f1': [0.0, 0.0, 0.0, 0.0, 0.0],
 'gmean': [0.0, 0.0, 0.0, 0.0, 0.0],
 'exe_time': [0.00791025161743164,
  0.0072443485260009766,
  0.00722503662109375,
  0.008031845092773438,
  0.008867025375366211],
 'y_pred': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1])}

In [89]:
# #temp 
for i in range(len(datasets)):
    if not os.path.isdir('./Results/'+datasets[i]):
        os.mkdir('./Results/'+datasets[i])
    for classifier in classifiers:
        if not os.path.isdir('./Results/'+datasets[i]+'/'+classifier):
            os.mkdir('./Results/'+datasets[i]+'/'+classifier)
            


In [90]:
def res_to_files(dataset, classifier, dict_res):
    np.save('./Results/'+dataset+'/'+classifier+'/mcc.npy',dict_res['mcc'])
    np.save('./Results/'+dataset+'/'+classifier+'/auc.npy',dict_res['auc'])
    np.save('./Results/'+dataset+'/'+classifier+'/f1.npy',dict_res['f1'])
    np.save('./Results/'+dataset+'/'+classifier+'/gmean.npy',dict_res['gmean'])
    np.save('./Results/'+dataset+'/'+classifier+'/exe_time.npy',dict_res['exe_time'])
    np.save('./Results/'+dataset+'/'+classifier+'/y_pred.npy',dict_res['y_pred'])
    

In [93]:
# for classifier in classifiers:
#     for dataset in datasets:
#         if not os.path.exists('./Results/'+dataset+'/y_pred.npy'):
#             print(classifier," => ", dataset, "=>", end=' ')
#             # tmp_res = clasify(dataset, classifier)
#             # print(tmp_res['auc'])
#             try:
#                 tmp_res = clasify(dataset, classifier)
#                 res_to_files(dataset, classifier, tmp_res)
#             except:
#                 print(dataset, classifier)


classifier = 'RUS'
for dataset in datasets:
    if not os.path.exists('./Results/'+dataset+'/y_pred.npy'):
        print(classifier," => ", dataset, "=>", end=' ')
        tmp_res = clasify(dataset, classifier)
        res_to_files(dataset, classifier, tmp_res)
        print(tmp_res['auc'])
        
        

RUS  =>  yeast-1_vs_7-5-fold => Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-1tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-2tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-3tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-4tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-5tra.xlsx
[0.5, 0.5, 0.5, 0.5, 0.5]
RUS  =>  led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold => Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-1tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-2tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-3tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-4tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-5tra.xlsx
[0.5, 0.5, 0.5, 0.5, 0.5]
RUS  =>  ecoli-0-6-7_vs_3-5-5-fold => Datasets/ecoli-0-6-7_vs_3-5-5-fold/ecoli-0-6-7_vs_3-5-5-1tra.xlsx
Datasets/ecoli-0-6-7_vs_3-5-5-fold/ecoli-0

ValueError: could not convert string to float: 'I'

In [68]:
2

2

In [69]:
troubs = {}
for dataset in datasets:
    for classifier in classifiers:
        files = glob.glob('./Results/'+dataset+'/'+classifier+'/*.npy')
        if len(files) != 6:
            if dataset in troubs:
                troubs[dataset].append(classifier)
            else:
                troubs[dataset] = [classifier]
            print(dataset, classifier)
        
print([dataset for dataset in troubs if len(troubs[dataset])==4])

yeast-1_vs_7-5-fold RUS
yeast-1_vs_7-5-fold SMOTE
yeast-1_vs_7-5-fold Ada1
yeast-1_vs_7-5-fold AdaCost
ecoli-0-1-4-6_vs_5-5-fold RUS
ecoli-0-1-4-6_vs_5-5-fold SMOTE
shuttle-c2-vs-c4-5-fold SMOTE
ecoli-0-1_vs_5-5-fold RUS
ecoli-0-1_vs_5-5-fold SMOTE
abalone19-5- RUS
abalone19-5- SMOTE
abalone19-5- Ada1
abalone19-5- AdaCost
ecoli-0-1-4-7_vs_5-6-5-fold RUS
ecoli-0-1-4-7_vs_5-6-5-fold SMOTE
ecoli-0-3-4-7_vs_5-6-5-fold RUS
ecoli-0-3-4-7_vs_5-6-5-fold SMOTE
ecoli-0-1-3-7_vs_2-6-5-fold SMOTE
glass-0-1-4-6_vs_2-5-fold RUS
glass-0-1-4-6_vs_2-5-fold SMOTE
abalone19-5-fold RUS
abalone19-5-fold SMOTE
abalone19-5-fold Ada1
abalone19-5-fold AdaCost
ecoli-0-4-6_vs_5-5-fold RUS
ecoli-0-4-6_vs_5-5-fold SMOTE
yeast-0-3-5-9_vs_7-8-5-fold RUS
yeast-0-3-5-9_vs_7-8-5-fold SMOTE
yeast-1-2-8-9_vs_7-5-fold RUS
['yeast-1_vs_7-5-fold', 'abalone19-5-', 'abalone19-5-fold']


In [101]:
# convert categorical variables into numerical
from sklearn.preprocessing import LabelEncoder
 
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label

base_path = 'Datasets/'
need_to_convert = ['abalone19-5-', 'abalone19-5-fold']
for needed in need_to_convert:
    for datFile in glob.glob(base_path+needed+'/*.dat'):
        print(datFile)
        df = read_dot_dat_file(datFile)
        label = le.fit_transform(df['Sex,'])        
        df['Sex,'] = label
        df.to_excel(base_path+needed+'/'+datFile.split('/')[-1].split('.')[0]+'.xlsx')
        break
        
        

Datasets/abalone19-5-/abalone19-5-5tst.dat
['{positive,', 'negative}']
     Sex,  Length,  Diameter,  Height,  Whole_weight,  Shucked_weight,  \
0       1    0.330      0.255    0.080         0.2050           0.0895   
1       2    0.475      0.370    0.125         0.5095           0.2165   
2       2    0.590      0.445    0.140         0.9310           0.3560   
3       2    0.665      0.525    0.165         1.3380           0.5515   
4       2    0.465      0.355    0.105         0.4795           0.2270   
..    ...      ...        ...      ...            ...              ...   
829     2    0.475      0.360    0.140         0.5135           0.2410   
830     0    0.585      0.475    0.165         1.0530           0.4580   
831     1    0.475      0.365    0.115         0.4990           0.2320   
832     0    0.515      0.400    0.125         0.6150           0.2865   
833     2    0.520      0.385    0.165         0.7910           0.3750   

     Viscera_weight,  Shell_weight  Clas

In [98]:
df


,"Sex,","Length,","Diameter,","Height,","Whole_weight,","Shucked_weight,","Viscera_weight,",Shell_weight,Class
0,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,-1
1,M,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.1650,-1
2,M,0.590,0.445,0.140,0.9310,0.3560,0.2340,0.2800,-1
3,M,0.665,0.525,0.165,1.3380,0.5515,0.3575,0.3500,-1
4,M,0.465,0.355,0.105,0.4795,0.2270,0.1240,0.1250,-1
...,...,...,...,...,...,...,...,...,...
829,M,0.475,0.360,0.140,0.5135,0.2410,0.1045,0.1550,-1
830,F,0.585,0.475,0.165,1.0530,0.4580,0.2170,0.3000,-1
831,I,0.475,0.365,0.115,0.4990,0.2320,0.0885,0.1560,-1
832,F,0.515,0.400,0.125,0.6150,0.2865,0.1230,0.1765,-1
